In [1]:
# imports
from pathlib import Path
from time import time
from math import ceil, log

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from utils import (
    read_single_problem_from_path_as_adjacency,
    check_validity_for_adjacency,
    transform_zero_one_clique_to_nodes,
    transform_zero_one_clique_to_nodes_of_base
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, ALL_BEST_KNOWN
from heuristics import (
    descending_degree_glutonous_heuristic,
)
from meta_heuristics import (
    ExploringMetaHeuristicRunner,
    ExploringTaboosMetaHeuristicRunner,
    ExploringTaboosRestartMetaHeuristicRunner,
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
# Simple run
instance = "C500.9.txt"

# Load instance
(
    number_of_nodes,
    number_of_edges,
    graph,
    degrees,
) = read_single_problem_from_path_as_adjacency(
    instance_path=OTHER_INSTANCES_DIR / instance
)
# Build starting clique
starting_clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
# Create runner
max_time = 30
max_intensification_without_improve = 5
max_exploring_iterations_without_improve = 3 * max_intensification_without_improve
max_restarts_iterations_without_improve = max_intensification_without_improve

runner = ExploringTaboosMetaHeuristicRunner(
    graph=graph,
    degrees=degrees,
    starting_clique=starting_clique,
    max_time=max_time,
    max_intensification_without_improve=max_intensification_without_improve,
    max_exploring_iterations_without_improve=max_exploring_iterations_without_improve,
    max_restarts_iterations_without_improve=max_restarts_iterations_without_improve,
)
(
    clique,
    time_best_found,
    iteration_of_best,
    number_of_iterations,
) = runner.meta_heuristic()
# Assert that clique is valid
assert check_validity_for_adjacency(graph, clique)
# Print clique in nodes
print(transform_zero_one_clique_to_nodes(clique=clique))
print(transform_zero_one_clique_to_nodes_of_base(clique=clique))


[4, 5, 20, 39, 41, 60, 76, 91, 96, 109, 146, 162, 178, 180, 185, 192, 193, 203, 206, 228, 229, 241, 243, 244, 248, 258, 267, 279, 289, 290, 294, 301, 306, 324, 326, 328, 330, 349, 350, 380, 399, 401, 404, 406, 410, 414, 441, 450, 455, 477, 487, 489, 496]
[5, 6, 21, 40, 42, 61, 77, 92, 97, 110, 147, 163, 179, 181, 186, 193, 194, 204, 207, 229, 230, 242, 244, 245, 249, 259, 268, 280, 290, 291, 295, 302, 307, 325, 327, 329, 331, 350, 351, 381, 400, 402, 405, 407, 411, 415, 442, 451, 456, 478, 488, 490, 497]


In [2]:
#  Run with display
columns = [
    "instance",
    "start",
    "end",
    "bound",
    "gap",
    "start/end diff",
    "time of best",
    "iteration_of_best",
    "total_iterations",
]
display_dataframe = pd.DataFrame({column: [] for column in columns})
max_time = 20
max_intensification_without_improve = 5
max_exploring_iterations_without_improve = 3 * max_intensification_without_improve
max_restarts_iterations_without_improve = max_intensification_without_improve
# Running
for instance in BASE_INSTANCES_FILES:
    print(instance)
    number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
        instance_path=BASE_INSTANCES_DIR / instance
    )
    starting_clique = descending_degree_glutonous_heuristic(
        graph=graph, degrees=degrees
    )

    max_taboos = min(10, ceil(np.sqrt(np.sum(starting_clique))))
    runner = ExploringTaboosMetaHeuristicRunner(
        graph=graph,
        degrees=degrees,
        starting_clique=starting_clique,
        max_time=max_time,
        max_intensification_without_improve=max_intensification_without_improve,
        max_exploring_iterations_without_improve=max_exploring_iterations_without_improve,
        max_restarts_iterations_without_improve=max_restarts_iterations_without_improve,
    )
    # For storing results
    runner.update_store_variables(store_cliques=True, storing_file=f"{instance}_taboos_results.txt")

    (
        clique,
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ) = runner.meta_heuristic()
    assert check_validity_for_adjacency(graph, clique)
    # Adding to display
    new_row = [
        instance,
        np.sum(starting_clique),
        np.sum(clique),
        ALL_BEST_KNOWN[instance],
        "{:.1%}".format(1 - np.sum(clique) / ALL_BEST_KNOWN[instance]),
        "{:.1%}".format(
            np.sum(np.where(starting_clique == clique, 0, 1)) / number_of_nodes
        ),
        time_best_found,
        iteration_of_best,
        number_of_iterations,
    ]
    display_dataframe.loc[len(display_dataframe)] = new_row

print(display_dataframe)


brock200_2.col
dsjc125.1.col
random-10.col
random-100.col
random-40.col
random-70.col
         instance  start  end  bound   gap start/end diff  time of best  \
0  brock200_2.col      7   11     12  8.3%           9.0%     17.874304   
1   dsjc125.1.col      4    4      4  0.0%           4.8%      0.000000   
2   random-10.col      5    5      5  0.0%           0.0%      0.000000   
3  random-100.col      8    9      9  0.0%          15.0%      0.048868   
4   random-40.col      7    7      7  0.0%          20.0%      0.000000   
5   random-70.col      7    8      8  0.0%          12.9%      0.020972   

   iteration_of_best  total_iterations  
0               1779              1978  
1                  0              5023  
2                  0             60830  
3                 11              4256  
4                  0             11530  
5                  9              5958  
